In [ ]:
# Aufgabe:
# 1. Webscraping
# a. Using an AI of your choice (e.g., ki.th-koeln.de), write a webscraper, that turns the table from
# https://pokemondb.net/pokedex/all into a pandas DataFrame.
# b. Bonus: Make your tool also retrieve and display the sprite images.
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URL
url = "https://pokemondb.net/pokedex/all"

# Fetch the page content
response = requests.get(url)
response.raise_for_status()  # Check if the request was successful
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table containing Pokémon data
table = soup.find('table', {'id': 'pokedex'})

# Extract the table headers and rows
headers = [th.text for th in table.find('thead').find_all('th')]
rows = table.find('tbody').find_all('tr')

# Extract the data
pokemon_data = []
for row in rows:
    columns = row.find_all('td')
    sprite_tag = row.find('td').find('span').find('img') if row.find('td') else None  # Fetch sprite image URL safely
    sprite_url = sprite_tag['src'] if sprite_tag else 'No Image'  # Default to 'No Image' if not found
    data = [col.text.strip() for col in columns]
    data.insert(1, sprite_url)  # Insert sprite URL after the Pokémon ID
    pokemon_data.append(data)

# Add sprite header to the list of headers
headers.insert(1, 'Sprite URL')

# Create a pandas DataFrame
df = pd.DataFrame(pokemon_data, columns=headers)

# Display the first few rows of the DataFrame
print(df.head())

      # Sprite URL                    Name          Type Total  HP Attack  \
0  0001   No Image               Bulbasaur  Grass Poison   318  45     49   
1  0002   No Image                 Ivysaur  Grass Poison   405  60     62   
2  0003   No Image                Venusaur  Grass Poison   525  80     82   
3  0003   No Image  Venusaur Mega Venusaur  Grass Poison   625  80    100   
4  0004   No Image              Charmander          Fire   309  39     52   

  Defense Sp. Atk Sp. Def Speed  
0      49      65      65    45  
1      63      80      80    60  
2      83     100     100    80  
3     123     122     120    80  
4      43      60      50    65  


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

# Aufgabenstellung:
# a. Was sind die stärksten Pokémon (von jedem Typ)?
# b. Was sind die besten Angreifer?
# c. Was sind die Durchschnittswerte der Stats für jeden Typ?
# d. [Eigene Ideen hier]

# Define the URL
url = "https://pokemondb.net/pokedex/all"

# Fetch the page content
response = requests.get(url)
response.raise_for_status()  # Check if the request was successful
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table containing Pokémon data
table = soup.find('table', {'id': 'pokedex'})

# Extract the table headers and rows
headers = [th.text.strip() for th in table.find('thead').find_all('th')]
rows = table.find('tbody').find_all('tr')

# Extract the data
pokemon_data = []
for row in rows:
    columns = row.find_all('td')
    
    # Extract the sprite image URL
    sprite_tag = columns[0].find('img') if len(columns) > 0 else None  # Get the image from the first column
    sprite_url = sprite_tag['src'] if sprite_tag else 'No Image'  # Default to 'No Image' if not found
    sprite_url = urljoin(url, sprite_url)  # Combine relative URL with the base URL
    
    # Extract other Pokémon data
    data = [col.text.strip() for col in columns]
    data.insert(1, sprite_url)  # Insert sprite URL after the Pokémon ID
    pokemon_data.append(data)

# Add sprite header to the list of headers
headers.insert(1, 'Sprite URL')

# Create a pandas DataFrame
df = pd.DataFrame(pokemon_data, columns=headers)

# Konvertiere numerische Spalten in den entsprechenden Datentyp
df['Total'] = pd.to_numeric(df['Total'])
df['HP'] = pd.to_numeric(df['HP'])
df['Attack'] = pd.to_numeric(df['Attack'])
df['Defense'] = pd.to_numeric(df['Defense'])
df['Sp. Atk'] = pd.to_numeric(df['Sp. Atk'])
df['Sp. Def'] = pd.to_numeric(df['Sp. Def'])
df['Speed'] = pd.to_numeric(df['Speed'])

# a. Die stärksten Pokémon (mit der höchsten Gesamtpunktzahl) pro Typ
strongest_per_type = df.loc[df.groupby('Type')['Total'].idxmax()][['Type', 'Name', 'Total']]
print(strongest_per_type)

# b. Die besten Angreifer (höchster Angriffswert)
best_attackers = df.loc[df['Attack'].idxmax()][['Name', 'Attack', 'Type']]
print(best_attackers)

# c. Durchschnittswerte der Stats pro Typ
avg_stats_per_type = df.groupby('Type')[['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']].mean()
print(avg_stats_per_type)

               Type                      Name  Total
167             Bug                    Pinsir    500
1095       Bug Dark                     Lokix    450
888    Bug Electric                  Vikavolt    500
896       Bug Fairy                  Ribombee    464
274    Bug Fighting  Heracross Mega Heracross    600
...             ...                       ...    ...
176       Water Ice                    Lapras    535
94     Water Poison                Tentacruel    515
108   Water Psychic      Slowbro Mega Slowbro    590
687      Water Rock                Carracosta    495
489     Water Steel                  Empoleon    530

[221 rows x 3 columns]
Name      Mewtwo Mega Mewtwo X
Attack                     190
Type          Psychic Fighting
Name: 201, dtype: object
                      HP      Attack     Defense     Sp. Atk     Sp. Def  \
Type                                                                       
Bug            48.920000   48.480000   54.960000   37.560000   46.0400

In [4]:
# Aufgabenstellung:
# BONUS: Baue deine eigene coole Pokédex-Anwendung mit einer grafischen Benutzeroberfläche unter Verwendung von KI!

import tkinter as tk
from tkinter import ttk
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Lade Daten von der Website
url = "https://pokemondb.net/pokedex/all"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', {'id': 'pokedex'})

# Extrahiere die Daten
pokemon_data = []
for row in table.find('tbody').find_all('tr'):
    columns = row.find_all('td')
    sprite_url = columns[0].find('img')['src'] if columns[0].find('img') else 'No Image'
    data = [col.text.strip() for col in columns]
    data.insert(1, sprite_url)
    pokemon_data.append(data)

# Erstelle DataFrame
df = pd.DataFrame(pokemon_data, columns=['ID', 'Sprite URL', 'Name', 'Type', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed'])

# Tkinter Fenster
root = tk.Tk()
root.title("Pokédex")

# Pokémon Dropdown
pokemon_combobox = ttk.Combobox(root, values=df['Name'].tolist(), width=50)
pokemon_combobox.grid(row=0, column=0, padx=10, pady=10)

# Anzeige von Pokémon-Daten
def show_details():
    pokemon = df[df['Name'] == pokemon_combobox.get()].iloc[0]
    name_label.config(text=f"Name: {pokemon['Name']}")
    type_label.config(text=f"Type: {pokemon['Type']}")
    total_label.config(text=f"Total: {pokemon['Total']}")
    hp_label.config(text=f"HP: {pokemon['HP']}")
    attack_label.config(text=f"Attack: {pokemon['Attack']}")

# Button zur Anzeige der Details
details_button = tk.Button(root, text="Show Details", command=show_details)
details_button.grid(row=1, column=0, padx=10, pady=10)

# Labels zur Anzeige von Pokémon-Daten
name_label = tk.Label(root, text="Name: ")
name_label.grid(row=2, column=0, padx=10, pady=5, sticky="w")

type_label = tk.Label(root, text="Type: ")
type_label.grid(row=3, column=0, padx=10, pady=5, sticky="w")

total_label = tk.Label(root, text="Total: ")
total_label.grid(row=4, column=0, padx=10, pady=5, sticky="w")

hp_label = tk.Label(root, text="HP: ")
hp_label.grid(row=5, column=0, padx=10, pady=5, sticky="w")

attack_label = tk.Label(root, text="Attack: ")
attack_label.grid(row=6, column=0, padx=10, pady=5, sticky="w")

# Starte die Tkinter Anwendung
root.mainloop()